# Nuggets creation and scoring

In [ ]:
from pyterrier_rag.backend import OpenAIBackend
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("casperhansen/llama-3.3-70b-instruct-awq")

generation_args={
    "temperature": 0.6,
    "max_tokens": 256,
}

# this could equally be a real OpenAI models
llama = OpenAIBackend(model_name, 
                      api_key=os.environ['IDA_LLM_API_KEY'],
                      generation_args=generation_args,
                      base_url="http://api.llm.apps.os.dcs.gla.ac.uk/v1", 
                      verbose=True, 
                      parallel=64)

In [ ]:
from fastchat.conversation import get_conv_template
from open_nuggetizer.nuggetizer import Nuggetizer

conv_template = get_conv_template("meta-llama-3.1-sp")

nuggetizer = Nuggetizer(
    backend=backend, 
    conversation_template=conv_template,
    verbose=True
)

nuggets = load_csv("nuggets.csv")
if nuggets is None:
    nuggets = nuggetizer.create(baseline)
    save_csv("nuggets.csv", nuggets)

scored_nuggets = load_csv("scored_nuggets.csv")
if scored_nuggets is None:
    scored_nuggets = nuggetizer.score(nuggets)
    save_csv("scored_nuggets.csv", scored_nuggets)

# Evaluation

In [ ]:
from pyterrier_rag.prompt import Concatenator
from pyterrier_rag.readers import Reader
from pyterrier_rag.prompt import PromptTransformer
from jinja2 import Template

def make_callable_template(template: Template):
    def template_call(**kwargs):
        return template.render(**kwargs)

    return template_call

GENERIC_PROMPT = Template(
    "Use the context information to answer the Question: \n Context: {{ context }} \n Question: {{ query }} \n Answer:"
)

prompt = PromptTransformer(
            instruction=make_callable_template(GENERIC_PROMPT),
            system_message="You are an helpful assistant.",
            conversation_template=conv_template,
            input_fields=[
                "qcontext",
                "query",
            ],
        )

reader = Reader(backend, prompt)
rag_pipeline = monoT5_ret % 3 >> Concatenator() >> reader

results = (rag_pipeline)(topics_df.head(2))
results

In [ ]:
scored_nuggets = scored_nuggets.rename(columns={"query_id": "qid"})
results = results.rename(columns={"query_id": "qid", "query_0": "query"})
for element in nuggetizer.VitalScore().iter_calc(scored_nuggets, results):
    print(f"Query ID: {element.query_id}, Measure: {element.measure}, Value: {element.value}")